## Publish LoRaWAN payloads to a myDevices Cayenne web dashboard

### Introduction

In the [LoRaWAN getting-started Notebook](../getting-started/lorawan-getting-started.ipynb), we learnt how to use LoRa boards with [CampusIoT Chirpstack](https://lns.campusiot.imag.fr). In addition to send/receive messages to/from the ChirpStack backend, it is also possible to configure [integrations](https://www.thethingsnetwork.org/docs/applications/integrations.html) to a LoRaWAN network. One of these integrations is using webhooks, which allows to retrieve and display sensors data in customizable dashboards. This is what is used by a Cayenne application to interact with the ChirpStack backend.

- The first step of this Notebook is to configure the integration between a ChirpStack and myDevices Cayenne. This is explained in the next section.
- The second step will be to implement the RIOT application. This is explained in the "Write your RIOT application" section.

### Configure the integration between a ChirpStack application and myDevices Cayenne

If you followed the **[LoRaWAN getting-started Notebook](../getting-started/lorawan-getting-started.ipynb)**,
the only thing that you have to do is to configure the integration of your ChirpStack application with
myDevices Cayenne: this is simple, go to the **INTEGRATIONS** tab, select
**CREATE** and the choose the **myDevices.com** integration and the **Cayenne** endpoint.

<figure style="text-align:center">
    <img src="images/chirpstack-integration-mydevices.png" width="640" align="center"/><br/><br/>
    <figcaption><em>Integrations - myDevices</em></figcaption>
</figure>

Go to the devices **CONFIGURATION** tab and switch the profile to **OTAA_CLASS_A_LPP**:

<figure style="text-align:center">
    <img src="images/chirpstack-profile-lpp.png" width="640" align="center"/><br/><br/>
    <figcaption><em>Integrations - myDevices</em></figcaption>
</figure>

Now uplink data will be forwarded to a myDevice endpoint.

### Register your device in myDevices

1. Create an account on [myDevices](https://mydevices.com/)
2. Log-in and click on LoRa
3. Select **ChirpStack** at the bottom of the left menu bar, click on Cayenne LPP
   and fill in your DevEUI.

<figure style="text-align:center">
    <img src="images/mydevices-register-device.png" width="640" align="center"/><br/><br/>
    <figcaption><em>myDevice Register Device</em></figcaption>
</figure>

### Write your RIOT application

myDevices Cayenne requires the devices to encode the messages in a specific payload format called **[Cayenne LPP](https://developers.mydevices.com/cayenne/docs/lora/#lora-cayenne-low-power-payload)**. This format is a binary format designed to reduce the payload size (remember that LoRaWAN has a very low datarate) while remaining quite flexible: the format encodes several channels that contain different kind of sendor data types (temperature, relative humidity, etc).

In the same directory as this notebook, you will find the 2 files of a basic RIOT application: `Makefile`, `main.c`. This application already contains code to read sensors, join a LoRaWAN and send a basic payload every 20s.

You can double click the files in the JupyterLab browser to open them in new tabs and start editing them.

As you can see, the application is already quite complete but the DEVEUI/APPEUI/APPKEY are empty (full of zeros) and the encoding to the Cayenne LPP format is missing in `main.c`. Fortunately, RIOT provides support for [a library for encoding to the Cayenne LPP](https://github.com/aabadie/cayenne-lpp). This is what will be used in this Notebook.

1. Change the DEVEUI, APPEUI, APPKEY default values (see C static arrays lines 28 to 30 in main.c). From your device **KEYS(OTAA)**
you can easily copy the Application Key as a hexarray (keep MSB representation).

For the device id you can use the following snippet to format the hexstring as required:

In [ ]:
deveui = bytearray.fromhex("949e3d7a45528de9")
print(', '.join(['{:0=#4x}'.format(x) for x in deveui]))

  **Important**: if you don't edit the deveui/appeui/appkey values, you device won't be able to join the network and send messages.

2. Add the missing Cayenne LPP header (under `/* TODO: Add the cayenne_lpp header */`):

```c
#include "cayenne_lpp.h"
```

3. Declare globally Cayenne LPP descriptor:

```c
static cayenne_lpp_t lpp;
```

4. In the sender function, prepare the cayenne lpp payload with the temperature measured value:

```c
cayenne_lpp_add_temperature(&lpp, 0, (float)temperature / 1000);
```

5. You have to replace the send of the dummy payload with the send of the cayenne lpp payload, e.g:

```
uint8_t ret = semtech_loramac_send(&loramac, lpp.buffer, lpp.cursor);
```

instead of

```
uint8_t ret = semtech_loramac_send(&loramac, (uint8_t *)message, strlen(message));
```

6. At the end of the while loop, you have to clear the cayenne lpp payload buffer between each send:

```c
cayenne_lpp_reset(&lpp);
```

7. You can now build and flash the application:

In [ ]:
!make flash


8. Open a terminal (use `File > New > Terminal`):

In [ ]:
make riot/polytech/chirpstackcayenne-lpp term


In the terminal, you should see you device join the network with success and start sending payload to ChirpStack.
If you check the details of the payload Uplink in your device **DEVICE DATA** tab
you should see the decoded temperature data.

<figure style="text-align:center">
    <img src="images/chirpstack-lpp-data.png" width="640" align="center"/><br/><br/>
    <figcaption><em>ChirpStack LPP decoded Data</em></figcaption>
</figure>

Your myDevices  dashboard should start populating with new sensors values!

<figure style="text-align:center">
    <img src="images/mydevices-dashboard.png" width="640" align="center"/><br/><br/>
    <figcaption><em>myDevice Dashboard Sensor Data</em></figcaption>
</figure>

### Going further

Modify the RIOT application to add the `bmp280` atmospheric pressure sensor values to the Cayenne LPP payload:
- You can have a look at the [online documentation of the driver](https://doc.riot-os.org/group__drivers__bmx280.html)
- You can have a look at the [test application code](https://github.com/RIOT-OS/RIOT/tree/master/tests/driver_bmx280) which provides driver API usage